In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pylab as plt
import math
import os

In [3]:
X_train = pd.read_csv(os.path.abspath("../input")+"/choi_num_features_train.csv" , encoding = 'utf-8')
X_test = pd.read_csv(os.path.abspath("../input")+"/choi_num_features_test.csv" , encoding = 'utf-8')

In [4]:
# 학습, 평가데이터 불러오기
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')
target = pd.read_csv(os.path.abspath("../input")+"/y_train.csv" , encoding = 'cp949').group
y_train = pd.read_csv(os.path.abspath("../input")+"/y_train.csv" , encoding = 'cp949')

In [5]:
data = pd.concat([train, test]).reset_index(drop=True)

In [10]:
data

,custid,sales_month,sales_day,sales_dayofweek,sales_time,str_nm,goodcd,brd_nm,corner_nm,pc_nm,part_nm,team_nm,buyer_nm,import_flg,tot_amt,dis_amt,net_amt,inst_mon,inst_fee
0,0,6,25,일,1212,무역점,2116050008000,에스티로더,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,90000,9000,81000,3,0
1,0,6,25,일,1242,무역점,4125440008000,시슬리,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,1,39000,3900,35100,1,0
2,0,8,26,토,1810,본점,2116052008000,크리니크,수입종합화장품,화장품,잡화파트,잡화가용팀,화장품,1,175000,17500,157500,3,0
3,0,8,26,토,1830,본점,4106430119900,듀퐁,수입의류,명품토탈,잡화파트,잡화가용팀,수입명품,1,455000,45500,409500,3,0
4,0,9,3,일,1802,무역점,2139141008000,랑콤,수입종합화장품,화장품,명품잡화,잡화가용팀,화장품,0,100000,10000,90000,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040034,49993,13,31,수,1750,신촌점,4405551020474,톰키드,아동,아동,아동문화,잡화가용팀,유아동복,0,20000,0,20000,1,0
1040035,49993,13,31,수,1833,신촌점,2139140008300,폴로화장품,향수,화장품,패션잡화,잡화가용팀,화장품,0,70000,3500,66500,1,0
1040036,49994,16,14,토,1750,본점,4230120011274,스테파넬,영트랜드,영트렌디,"케주얼,구두,아동",의류패션팀,영캐주얼,0,39000,0,39000,1,0
1040037,49994,16,14,토,1810,본점,4409271026010,써스데이아일앤드,진케주얼,진케주얼,"케주얼,구두,아동",의류패션팀,유니캐주얼,0,34200,0,34200,1,0


In [13]:
data.goodcd.nunique()
data.brd_nm.nunique()
data.buyer_nm.nunique()

35

In [9]:
#%%writefile word2vec_goodcd.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')

train['goodcd']=train['goodcd'].astype('str')
test['goodcd']=test['goodcd'].astype('str')

### Make corpus
p_level = 'goodcd'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))  # 복원추출
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))
### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)

C:\Users\choij\AppData\Local\Temp\ipykernel_2892\3679107954.py:47: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


In [8]:
len(train_corpus[1]

['4400213022910',
 '4502161930200',
 '2116052008000',
 '4309200019900',
 '4309200019900',
 '4309200019900',
 '4134870003200',
 '4309200019900',
 '4502161930200',
 '4301000017000',
 '4400213022910',
 '4522862046000',
 '2116052008000',
 '2116052008000',
 '4400213022910',
 '4301000017000',
 '4134870003200',
 '4522862046000',
 '4522862046000',
 '2116052008000',
 '4134870003200',
 '4502161930200',
 '4134870003200',
 '4502161930200',
 '2116052008000',
 '4134870003200',
 '4400213022910',
 '4309200019900',
 '2116052008000',
 '4309200019900',
 '4502161930200',
 '4522862046000',
 '2116052008000',
 '4301000017000',
 '4309200019900',
 '2116052008000',
 '4134870003200',
 '4309200019900',
 '4502161930200',
 '4309200019900',
 '4309200019900',
 '4522862046000',
 '2116052008000',
 '4134870003200',
 '4134870003200',
 '4134870003200',
 '2116052008000',
 '4301000017000',
 '4400213022910',
 '4309200019900',
 '4522862046000',
 '4309200019900',
 '4301000017000',
 '4400213022910',
 '4301000017000',
 '21160520

## goodcd

In [13]:
%%writefile word2vec_goodcd.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')

train['goodcd']=train['goodcd'].astype('str')
test['goodcd']=test['goodcd'].astype('str')

### Make corpus
p_level = 'goodcd'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))  # 복원추출
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                # 해당 단어가 있으면, max 뽑기.
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_goodcd = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_goodcd = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Overwriting word2vec_goodcd.py


In [14]:
%run word2vec_goodcd.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_goodcd.py:48: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## corner_nm

In [15]:
%%writefile word2vec_corner.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')


### Make corpus
p_level = 'corner_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))  # 복원추출
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_corner = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_corner = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Writing word2vec_corner.py


In [16]:
%run word2vec_corner.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_corner.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## brd_nm

In [17]:
%%writefile word2vec_brd.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')


### Make corpus
p_level = 'brd_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 300 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_brd = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_brd = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Writing word2vec_brd.py


In [18]:
%run word2vec_brd.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_brd.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## pc_nm

In [19]:
%%writefile word2vec_pc.py

### Imports
import pandas as pd
import numpy as np
import os



### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')


### Make corpus
p_level = 'pc_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 50 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_pc = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_pc = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Writing word2vec_pc.py


In [20]:
%run word2vec_pc.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_pc.py:47: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## part_nm

In [21]:
%%writefile word2vec_part.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')


### Make corpus
p_level = 'part_nm'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_part = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_part = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Writing word2vec_part.py


In [22]:
%run word2vec_part.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_part.py:46: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


## customer_info

In [23]:
%%writefile word2vec_customer_info.py

### Imports
import pandas as pd
import numpy as np
import os


### Read data
train = pd.read_csv(os.path.abspath("../input")+"/X_train.csv" , encoding = 'cp949')
test = pd.read_csv(os.path.abspath("../input")+"/X_test.csv" , encoding = 'cp949')
train['customer_info'] = train['brd_nm'].astype(str) + '_' + train['corner_nm'].astype(str) + '_' + train['pc_nm'].astype(str) + '_' + train['part_nm'].astype(str) + '_' + train['str_nm'].astype(str) + '_' + train['team_nm'].astype(str) + '_' + train['buyer_nm'].astype(str)
test['customer_info'] = test['brd_nm'].astype(str) + '_' + test['corner_nm'].astype(str) + '_' + test['pc_nm'].astype(str) + '_' + test['part_nm'].astype(str) + '_' + test['str_nm'].astype(str) + '_' + test['team_nm'].astype(str) + '_' + test['buyer_nm'].astype(str)

### Make corpus
p_level = 'customer_info'  # 상품 분류 수준

# W2V 학습데이터가 부족하여 구매한 상품 목록으로부터 n배 oversampling을 수행
def oversample(x, n, seed=0):
    if n == 0:
        return list(x)
    uw = np.unique(x)
    bs = np.array([])
    np.random.seed(seed)
    for j in range(n):
        bs = np.append(bs, np.random.choice(uw, len(uw), replace=True))
    return list(bs)

train_corpus = list(train.groupby('custid')[p_level].agg(oversample, 20))
test_corpus = list(test.groupby('custid')[p_level].agg(oversample, 20))


### Training the Word2Vec model
num_features = 100 # 단어 벡터 차원 수
min_word_count = 1 # 최소 단어 수
context = 5 # 학습 윈도우(인접한 단어 리스트) 크기

# 초기화 및 모델 학습
from gensim.models import word2vec

# 모델 학습
w2v = word2vec.Word2Vec(train_corpus, 
                        vector_size=num_features, 
                        min_count=min_word_count,
                        window=context,
                        seed=0, workers=1)
# 필요없는 메모리 unload
w2v.init_sims(replace=True)


### Make features
# 구매상품에 해당하는 벡터의 평균/최소/최대 벡터를 feature로 만드는 전처리기
class EmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = num_features
    def fit(self, X):
        return self
    def transform(self, X):
        return np.array([
            np.hstack([
                np.max([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.min([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),
                np.mean([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0),                
                #np.std([self.word2vec[w] for w in words if w in self.word2vec] or [np.zeros(self.dim)], axis=0)                
            ]) 
            for words in X
        ]) 

# W2V 기반 feature 생성
train_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).fit(train_corpus).transform(train_corpus))
test_features = pd.DataFrame(EmbeddingVectorizer(w2v.wv).transform(test_corpus))

train_features.columns = ['v'+f'{c+1:03d}' for c in train_features.columns]
test_features.columns = ['v'+f'{c+1:03d}' for c in test_features.columns]

# 학습용과 제출용 데이터로 분리
X_train_customer_info = pd.concat([pd.DataFrame({'custid': np.sort(train['custid'].unique())}), train_features], axis=1)#.to_csv('X_train_buyer.csv', index=False)
X_test_customer_info = pd.concat([pd.DataFrame({'custid': np.sort(test['custid'].unique())}), test_features], axis=1)#.to_csv('X_test_buyer.csv', index=False)

Writing word2vec_customer_info.py


In [24]:
%run word2vec_customer_info.py

C:\Users\choij\KMU_2022\머신러닝\Kaggle\머러뿌시조!\준용\notebook\word2vec_customer_info.py:47: DeprecationWarning: Call to deprecated `init_sims` (Gensim 4.0.0 implemented internal optimizations that make calls to init_sims() unnecessary. init_sims() is now obsoleted and will be completely removed in future versions. See https://github.com/RaRe-Technologies/gensim/wiki/Migrating-from-Gensim-3.x-to-4).
  w2v.init_sims(replace=True)


In [25]:
train['custid'].nunique()

21587

In [26]:
X_train_goodcd

,custid,v001,v002,v003,v004,v005,v006,v007,v008,v009,...,v291,v292,v293,v294,v295,v296,v297,v298,v299,v300
0,0,0.260338,0.186900,0.168903,0.237015,0.082573,0.113325,0.138312,0.078962,0.155916,...,-0.025991,-0.000108,0.000129,0.030848,-0.014227,0.024762,0.023667,0.003629,0.008228,0.018825
1,2,0.084522,0.169908,0.307696,0.089508,0.151550,0.133955,0.152679,0.070408,0.058381,...,-0.065100,-0.061458,0.035100,0.011991,-0.005212,0.048923,0.034615,0.059435,-0.026859,-0.007535
2,3,0.131593,0.108714,0.194974,0.115930,0.177746,0.160802,0.094236,0.251144,0.243313,...,-0.006330,-0.013183,0.032457,-0.035272,-0.025075,0.005341,-0.001299,-0.011132,-0.068903,-0.006227
3,4,0.045032,0.074991,0.099207,-0.014526,0.101435,0.197985,0.242388,0.013959,0.099826,...,0.056549,-0.091408,0.101024,0.066511,-0.019385,0.082454,0.040006,0.035480,0.005565,0.055077
4,5,0.135634,0.169908,0.307696,0.142087,0.149320,0.216524,0.121142,0.229865,0.205705,...,-0.035143,-0.023590,0.035310,0.060995,-0.041829,0.036085,0.060329,0.036049,-0.021792,-0.072991
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,0.227528,0.135506,0.243854,0.163731,0.165786,0.257303,0.142808,0.218661,0.212525,...,-0.044956,-0.001147,0.028618,-0.010848,-0.038713,0.073433,-0.000012,0.047249,0.015561,0.042151
21583,29996,0.099166,0.118184,0.178831,0.114362,0.103431,0.115233,0.101758,0.116661,0.204764,...,-0.004586,0.081517,0.034906,0.045792,-0.106703,-0.004214,0.051520,0.011508,-0.005477,0.060011
21584,29997,0.197128,0.013331,0.223647,0.137698,0.237140,0.143406,0.168600,0.120567,0.115442,...,-0.039707,0.013380,-0.013323,-0.019012,-0.088614,0.000520,0.001611,0.015674,0.040686,0.010941
21585,29998,0.125044,-0.020529,0.168903,0.237015,0.093186,0.205842,0.208659,0.111696,0.155916,...,0.014070,-0.002629,0.081760,0.003381,-0.029690,-0.049138,-0.023227,-0.031093,-0.021718,0.003012


In [27]:
X_train_corner

,custid,v001,v002,v003,v004,v005,v006,v007,v008,v009,...,v291,v292,v293,v294,v295,v296,v297,v298,v299,v300
0,0,0.056763,0.204751,0.149943,0.199081,0.008155,0.126571,0.242940,0.013867,0.059911,...,-0.034173,-0.019983,-0.027080,-0.107699,-0.010033,-0.056021,-0.080666,0.151612,-0.083945,0.038805
1,2,0.191025,0.204751,0.233470,0.199081,0.041457,0.155885,0.242940,0.013867,0.016937,...,0.009130,0.018982,-0.023767,-0.030189,-0.038454,0.003262,-0.115047,0.042662,-0.039757,0.051180
2,3,0.164620,0.204751,0.242797,0.199081,0.165624,0.221735,0.242940,0.105742,0.277712,...,0.029035,0.013338,-0.014734,-0.065919,-0.046983,0.005148,-0.039297,0.028133,-0.037585,-0.010387
3,4,0.098454,0.204751,-0.020919,0.199081,0.126295,0.072836,0.242940,0.010505,-0.008190,...,0.002299,0.127778,0.003761,0.036180,0.042810,-0.023432,0.036819,-0.006410,0.003924,0.092997
4,5,0.191025,0.204751,0.233470,0.199081,0.184796,0.204762,0.242940,0.170309,0.230378,...,0.002972,0.024026,0.009855,-0.024187,-0.079932,0.013594,-0.019925,0.048558,-0.027546,0.004001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21582,29995,0.097922,0.204751,0.178525,0.225589,0.184796,0.204762,0.242940,0.180570,0.116556,...,0.057885,0.021596,-0.069201,0.019445,0.014048,-0.007725,-0.019666,0.062189,-0.078740,0.003395
21583,29996,0.146767,0.174014,0.008641,0.183424,0.105979,0.184321,0.182815,0.068403,0.122801,...,0.010826,-0.005720,-0.071507,0.024601,0.061454,0.081770,0.018181,-0.030932,-0.050655,0.010133
21584,29997,0.105734,0.204751,0.154300,0.199081,0.147673,0.086705,0.242940,0.170309,0.155350,...,0.035925,0.012268,-0.030683,0.000322,-0.012866,-0.036804,-0.010471,0.010659,0.022828,-0.001081
21585,29998,0.148536,0.204751,-0.028444,0.199081,0.104160,0.126571,0.242940,0.045759,-0.008885,...,0.025044,0.005114,-0.009168,-0.086843,-0.013875,0.054422,-0.060501,-0.035885,-0.062862,0.014777


# Feature Merge

In [28]:
del X_train_goodcd['custid']
del X_test_goodcd['custid']
del X_train_corner['custid']
del X_test_corner['custid']
del X_train_brd['custid']
del X_test_brd['custid']
del X_train_pc['custid']
del X_test_pc['custid']
del X_train_part['custid']
del X_test_part['custid']
del X_train_customer_info['custid']
del X_test_customer_info['custid']

In [29]:
X_train_goodcd.columns = X_train_corner.columns.map(lambda x : "goodcd_" + str(x))
X_test_goodcd.columns = X_test_corner.columns.map(lambda x : "goodcd_" + str(x))
X_train_corner.columns = X_train_corner.columns.map(lambda x : "corner_" + str(x))
X_test_corner.columns = X_test_corner.columns.map(lambda x : "corner_" + str(x))
X_train_brd.columns = X_train_brd.columns.map(lambda x : "brd_" + str(x))
X_test_brd.columns = X_test_brd.columns.map(lambda x : "brd_" + str(x))
X_train_pc.columns = X_train_pc.columns.map(lambda x : "pc_" + str(x))
X_test_pc.columns = X_test_pc.columns.map(lambda x : "pc_" + str(x))
X_train_part.columns = X_train_part.columns.map(lambda x : "part_" + str(x))
X_test_part.columns = X_test_part.columns.map(lambda x : "part_" + str(x))
X_train_customer_info.columns = X_train_customer_info.columns.map(lambda x : "customer_info_" + str(x))
X_test_customer_info.columns = X_test_customer_info.columns.map(lambda x : "customer_info_" + str(x))

In [30]:
w2v_features_train = pd.concat([X_train_goodcd,X_train_corner, X_train_brd, X_train_pc, X_train_part, X_train_customer_info], axis=1) ; w2v_features_train
w2v_features_test = pd.concat([X_test_goodcd,X_test_corner, X_test_brd, X_test_pc, X_test_part, X_test_customer_info], axis=1) ; w2v_features_test

,goodcd_v001,goodcd_v002,goodcd_v003,goodcd_v004,goodcd_v005,goodcd_v006,goodcd_v007,goodcd_v008,goodcd_v009,goodcd_v010,...,customer_info_v291,customer_info_v292,customer_info_v293,customer_info_v294,customer_info_v295,customer_info_v296,customer_info_v297,customer_info_v298,customer_info_v299,customer_info_v300
0,0.129422,0.186900,0.307696,0.103418,0.165786,0.224780,0.197644,0.220256,0.222983,0.198732,...,-0.037652,-0.001466,-0.020826,-0.055512,0.017100,-0.006132,0.055545,0.009020,-0.011972,0.072295
1,0.171728,0.141858,0.242991,0.224061,0.142499,0.235643,0.220895,0.165577,0.207531,0.288024,...,-0.029072,0.011011,-0.014196,-0.060104,-0.038520,0.017734,0.006911,0.016092,-0.016071,0.006630
2,0.123039,0.083569,0.081921,0.174803,0.178339,0.205842,0.128738,0.217661,0.247399,0.194495,...,0.011028,-0.091674,-0.007643,-0.009296,0.020878,-0.008030,0.048004,-0.000222,-0.034725,0.027391
3,0.099166,-0.214820,-0.175486,0.071684,0.081226,-0.032194,-0.064322,0.060789,0.057255,-0.033473,...,-0.002563,-0.127233,-0.027754,-0.009906,0.022087,0.042219,0.092757,-0.097661,-0.106951,0.113660
4,0.062828,-0.007628,0.018975,0.037392,0.188506,0.089032,0.178485,0.138662,0.158986,0.054364,...,0.025815,-0.056631,0.014909,0.003187,-0.015764,-0.047946,-0.028684,0.035731,0.004185,0.040479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14375,0.061392,-0.059110,0.145619,0.136968,-0.039073,0.172800,0.054909,0.155830,0.205705,0.064942,...,0.010387,0.102729,-0.155586,0.047136,-0.024409,0.066448,0.092307,0.037059,0.009746,-0.008026
14376,-0.034577,-0.252133,-0.117986,-0.062228,0.067351,-0.176212,0.031558,-0.002536,0.058381,0.010223,...,0.108780,-0.118620,-0.002562,-0.051260,-0.110725,0.010442,0.094125,-0.016829,0.038454,0.060730
14377,-0.051799,-0.047219,0.081804,-0.110591,0.082573,0.006183,-0.117187,0.018908,0.155342,0.100011,...,0.000037,0.100021,-0.043835,0.016731,-0.066196,0.069820,-0.024159,0.078777,0.060873,0.000105
14378,0.137935,0.093982,0.035913,0.101195,0.080409,0.069784,0.093307,0.003883,0.047154,0.063743,...,0.001007,-0.077669,-0.086390,-0.069017,0.078894,0.092026,0.018246,-0.046485,0.066622,0.096271


In [31]:
w2v_features_train.to_csv('choi_w2v_features_train.csv', index=False)
w2v_features_test.to_csv('choi_w2v_features_test.csv', index=False)

***